## Import libraries

In [24]:
import pandas as pd
import urllib.request
import zipfile


### Download the results file from the TSE Dados Abertos

Uncomment when running the code for the first time, then you can comment again

In [25]:
# url_download = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2022_BR.zip'
# opener = urllib.request.build_opener()
# opener.addheaders = [('User-agent', 'Mozilla/5.0')]
# urllib.request.install_opener(opener)
# urllib.request.urlretrieve(url_download, f"CSV/votacao_secao_2022_BR.zip")

If you prefer you can download the files manually: The full list of files can be found [here](https://dadosabertos.tse.jus.br/dataset/resultados-2022)!

### Unzip the downloaded file


Uncomment when running the code for the first time, then you can comment again

In [26]:
# with zipfile.ZipFile(f"CSV/votacao_secao_2022_BR.zip", 'r') as zip_ref:
#     zip_ref.extractall("CSV/")

### Read the extracted CSV

In [27]:
df = pd.read_csv("CSV/votacao_secao_2022_BR.csv", encoding="Latin 1", sep=";", usecols=['NR_TURNO','SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','DS_CARGO','NM_VOTAVEL','QT_VOTOS'])

In [48]:
list_states = df['SG_UF'].unique()
# df['SG_UF'].unique()
df_ne = df[df['SG_UF'].isin(['BA','MA','PI','RN','PB','SE','CE','PE','AL'])]

In [49]:
# Quality check, total votes for Lula in Bahia
# df[(df['SG_UF']=='BA')&(df['NM_VOTAVEL']=='LUIZ INÁCIO LULA DA SILVA')&(df['DS_CARGO']=='PRESIDENTE')&(df['NR_TURNO']==2)]['QT_VOTOS'].sum()

### Filter by 2o turno and exclude null and branco votes, 

In [50]:
df_ne = df_ne[(df_ne['NR_TURNO']==2)].copy()

## RUN PIVOT

In [51]:
results_ne = df_ne[['SG_UF','NM_MUNICIPIO','NM_VOTAVEL','QT_VOTOS']].groupby(by=['SG_UF','NM_MUNICIPIO','NM_VOTAVEL']).sum("QT_VOTOS")
pivot_ne = results_ne.unstack(level='NM_VOTAVEL')
pivot_ne

QT_VOTOS                            \
NM_VOTAVEL                   JAIR MESSIAS BOLSONARO LUIZ INÁCIO LULA DA SILVA   
SG_UF NM_MUNICIPIO                                                              
AL    ANADIA                                   2019                      6166   
      ARAPIRACA                               54674                     61009   
      ATALAIA                                  7544                     13340   
      BARRA DE SANTO ANTÔNIO                   3586                      5155   
      BARRA DE SÃO MIGUEL                      2871                      2699   
...                                             ...                       ...   
SE    SÃO MIGUEL DO ALEIXO                      701                      2273   
      TELHA                                     900                      1925   
      TOBIAS BARRETO                           9726                     22390   
      TOMAR DO GERU                            2086                      4993   
      UMBAÚBA                                  4115                      9930   

                                                    
NM_VOTAVEL                   VOTO BRANCO VOTO NULO  
SG_UF NM_MUNICIPIO                                  
AL    ANADIA                         110       571  
      ARAPIRACA                     1726      6289  
      ATALAIA                        326      1412  
      BARRA DE SANTO ANTÔNIO         112       532  
      BARRA DE SÃO MIGUEL             61       286  
...                                  ...       ...  
SE    SÃO MIGUEL DO ALEIXO            22       125  
      TELHA                           24       103  
      TOBIAS BARRETO                 328      1046  
      TOMAR DO GERU                  127       382  
      UMBAÚBA                        243       552  

[1794 rows x 4 columns]

In [52]:
results_ne_sessao = df_ne[['SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','NM_VOTAVEL','QT_VOTOS']].groupby(by=['SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','NM_VOTAVEL']).sum("QT_VOTOS")
pivot_ne_sessao = results_ne_sessao.unstack(level='NM_VOTAVEL')
pivot_ne_sessao

QT_VOTOS  \
NM_VOTAVEL                          JAIR MESSIAS BOLSONARO   
SG_UF NM_MUNICIPIO NR_ZONA NR_SECAO                          
AL    ANADIA       48      90                         44.0   
                           91                         50.0   
                           92                         61.0   
                           93                         31.0   
                           94                         29.0   
...                                                    ...   
SE    UMBAÚBA      35      148                        30.0   
                           152                        76.0   
                           154                       100.0   
                           155                        36.0   
                           158                        71.0   

                                                                           \
NM_VOTAVEL                          LUIZ INÁCIO LULA DA SILVA VOTO BRANCO   
SG_UF NM_MUNICIPIO NR_ZONA NR_SECAO                                         
AL    ANADIA       48      90                           181.0         NaN   
                           91                           170.0         1.0   
                           92                           163.0         3.0   
                           93                           136.0         4.0   
                           94                           116.0         2.0   
...                                                       ...         ...   
SE    UMBAÚBA      35      148                           98.0         4.0   
                           152                          134.0         4.0   
                           154                          163.0         3.0   
                           155                          171.0         2.0   
                           158                          151.0         6.0   

                                               
NM_VOTAVEL                          VOTO NULO  
SG_UF NM_MUNICIPIO NR_ZONA NR_SECAO            
AL    ANADIA       48      90            25.0  
                           91            25.0  
                           92            11.0  
                           93             7.0  
                           94            13.0  
...                                       ...  
SE    UMBAÚBA      35      148            4.0  
                           152            8.0  
                           154            7.0  
                           155            7.0  
                           158            3.0  

[132578 rows x 4 columns]

## Export PIVOT

In [53]:
pivot_ne_sessao.to_excel("Output/pivot_ne_sessao_2turno.xlsx", merge_cells=False)

In [34]:
# df_results = df_ne[['SG_UF','NM_MUNICIPIO']].copy()
# df_results['Lula_abslt'] = ""
# df_results['Bolsonaro_abslt'] = ""
# df_results['Lula_pctg'] = ""
# df_results['Bolsonaro_pctg'] = ""
# df_results['VotosTotais'] = ""
# df_results.drop_duplicates(inplace=True)
# df_results = df_results.sort_values(by='SG_UF')
# df_results

In [35]:
# for i,r in df_results.iterrows():
#     print(r['SG_UF'],r['NM_MUNICIPIO'])
#     uf = r['SG_UF']
#     muni = r['NM_MUNICIPIO']
#     results = df_ne[(df_ne['SG_UF']==uf)&(df_ne['NM_MUNICIPIO']==muni)][['NM_VOTAVEL','QT_VOTOS']].groupby(by='NM_VOTAVEL').sum("QT_VOTOS")
#     results['pctg'] = (results.QT_VOTOS / results.QT_VOTOS.sum())*100
#     df_results.at[i,'Lula_pctg'] = results[results.index=='LUIZ INÁCIO LULA DA SILVA']['pctg'][0]
#     df_results.at[i,'Lula_abslt'] = results[results.index=='LUIZ INÁCIO LULA DA SILVA']['QT_VOTOS'][0]
#     bolsonaro_pctg = results[results.index=='JAIR MESSIAS BOLSONARO']['pctg'][0]
#     bolsonaro_abslt = results[results.index=='JAIR MESSIAS BOLSONARO']['QT_VOTOS'][0]    
#     df_results.at[i,'Bolsonaro_pctg'] = bolsonaro_pctg
#     df_results.at[i,'Bolsonaro_abslt'] = bolsonaro_abslt
#     df_results.at[i,'VotosTotais'] = results.QT_VOTOS.sum()

In [36]:
# df_results.sort_values(by='Bolsonaro_abslt')

### Check specific Sessao and Zona

In [37]:
zona = 195
secao = 76
check_secao = df_ne[(df_ne['NR_ZONA']==zona)&(df_ne['NR_SECAO']==secao)][['NM_VOTAVEL','QT_VOTOS']].groupby(by='NM_VOTAVEL').sum("QT_VOTOS")
df_ne[(df_ne['NR_ZONA']==zona)&(df_ne['NR_SECAO']==secao)][['SG_UF','NM_MUNICIPIO','NM_VOTAVEL','QT_VOTOS']]

,SG_UF,NM_MUNICIPIO,NM_VOTAVEL,QT_VOTOS
4024778,BA,PILÃO ARCADO,LUIZ INÁCIO LULA DA SILVA,65
